# Quellen: 

[Google Github, o. D.] - Google (o. D.). GitHub - google/sentencepiece: Unsupervised text tokenizer for Neural Network-based text generation. GitHub. [online] https://github.com/google/sentencepiece [abgerufen am 16.01.2022].

[Jitsi, o. D.] - Jitsi (o. D.). GitHub - jitsi/jiwer: Evaluate your speech-to-text system with similarity measures such as word error rate (WER). GitHub. [online] https://github.com/jitsi/jiwer [abgerufen am 17.01.2022].

[Klein et al., 2018] - Klein, Guillaume/Kim, Yoon/Deng, Yuntian/Nguyen, Vincent/Senellart, Jean/Rush, Alexander M. (2018). OpenNMT: Neural Machine Translation Toolkit. In: Proceedings of AMTA 2018, Vol. 1, MT Research Track, 177-184. [online] https://aclanthology.org/W18-1817.pdf [abgerufen am 30.12.21].

[Koehn, 2005] - Koehn, Philipp (2005). Europarl: A parallel corpus for statistical machine translation. In: MT Summit, 5, 79-86. [online] https://homepages.inf.ed.ac.uk/pkoehn/publications/europarl-mtsummit05.pdf [abgerufen am 04.01.2022]

[Mjpost, o. D.] - Mjpost (o. D.). GitHub - mjpost/sacrebleu: Reference BLEU implementation that auto-downloads test sets and reports a version string to facilitate cross-lab comparisons. GitHub. [online] https://github.com/mjpost/sacrebleu [abgerufen am 17.01.2022].

[OpenNMT, o. D.a] - Train — OpenNMT-py documentation (o. D.). OpenNMT. [online] https://opennmt.net/OpenNMT-py/options/train.html [abgerufen am 01.01.2022].

[OpenNMT, o. D.b] - OpenNMT (o. D.). GitHub - OpenNMT/OpenNMT-py: Open Source Neural Machine Translation in PyTorch. GitHub. [online] https://github.com/OpenNMT/OpenNMT-py [abgerufen am 04.01.2022].

[OpenNMT, o. D.c] - Translation — OpenNMT-py documentation (o. D.). OpenNMT. [online] https://opennmt.net/OpenNMT-py/examples/Translation.html [abgerufen am 01.01.2022].

[OPUS, o. D.] - Europarl (o. D.). OPUS. [online] https://opus.nlpl.eu/Europarl.php [abgerufen am 01.01.2022].

[Tiedemann, 2012] - Tiedemann, Jörg (2012). Parallel Data, Tools and Interfaces in OPUS. In: Proceedings of the 8th International Conference on Language Resources and Evaluation (LREC 2012). [online] http://www.lrec-conf.org/proceedings/lrec2012/pdf/463_Paper.pdf [abgerufen am 04.01.2022].

[Ymoslem, o. D.a] - Ymoslem (o. D.). GitHub - ymoslem/MT-Preparation: Machine Translation (MT) Preparation Scripts. GitHub. [online] https://github.com/ymoslem/MT-Preparation [abgerufen am 01.01.2022].

[Ymoslem, o. D.b] - Ymoslem (o. D.). GitHub - ymoslem/MT-Evaluation: Machine Translation (MT) Evaluation Scripts. GitHub. [online] https://github.com/ymoslem/MT-Evaluation [abgerufen am 04.01.2022].

In [ ]:
# Installiert OpenNMT-PyTorch
!pip install OpenNMT-py

In [ ]:
# Installiert weitere Pakete, die für das Experiment benötigt werden.
!pip3 install --user -r requirements.txt

In [ ]:
# Filtering: Datenbereinigung
!python3 filter.py Data/Europarl.de-en.en Data/Europarl.de-en.de en de

In [ ]:
# Teilwortmodelle (Subwording Models): Es müssen zwei Teilwortmodelle erstellt werden, um das Vokabular der Quell- und Zielsprache zu lernen.
!python3 train.py Data/Europarl.de-en.en-filtered.en Data/Europarl.de-en.de-filtered.de

In [ ]:
# Tokenisierung: In diesem Schritt werden die Modelle verwendet, die im vorherigen Schritt erzeugt wurden, um das Subwording der Quell- und Zieldateien für die maschinelle Übersetzung durchzuführen
!python3 subword.py source.model target.model Data/Europarl.de-en.en-filtered.en Data/Europarl.de-en.de-filtered.de

In [ ]:
# Aufteilung des parallelen Datensatzes in Trainings-, Validierungs- und Testdatensätze für die maschinelle Übersetzung.
!python3 train_dev_test_split.py 10000 10000 Data/Europarl.de-en.en-filtered.en.subword Data/Europarl.de-en.de-filtered.de.subword

In [ ]:
# Vokabular aufbauen. Mit -n_sample -1 das Vokabular über den gesamten Korpus berechnen.
!onmt_build_vocab -config europarl.yaml -n_sample -1

In [ ]:
# Training starten.
!onmt_train -config europarl.yaml

In [ ]:
# Training ab dem ausgewähltem Checkpoint fortsetzen.
!onmt_train -config europarl.yaml -train_from NMT-Model/model_baseline_step_40000.pt -continue

In [ ]:
# Versteckten .trash Ordner auf Paperspace - Gradient löschen
# !du --help
!rm -rf .Trash-0* # Müll entfernen
!du -sch .[!.]** |sort -h # Versteckte Ordner anzeigen

In [ ]:
# Übersetzen mit trainiertem Modell.
!python translate.py -model NMT-Model/model_baseline_step_40000.pt -src Data/Europarl.de-en.en-filtered.en.subword.test -output Output/pred.txt -replace_unk -verbose

In [ ]:
# Detokenisierung der maschinellen Übersetzung.
# Dieser Schritt ist sinnvoll nachdem das maschinelle Übersetzungsmodell trainiert und Dateien damit übersetzt wurden. Die generierten Zieldateien (d. h. die übersetzten Dateien) müssen detokenisiert werden.
!python3 desubword.py target.model Output/pred_batch_size_2k_60k.txt

In [ ]:
# Detokenisierung der Referenz.
!python3 desubword.py target.model Data/Europarl.de-en.de-filtered.de.subword.test

In [ ]:
# Evaluierung mit BLEU.
!python3 compute-bleu.py Data/Europarl.de-en.de-filtered.de.subword.test.desubword Output/pred_batch_size_2k_60k.txt.desubword

In [ ]:
# Evaluierung mit METEOR
# Bitte beachten, dass METEOR nur auf Satzebene und nicht auf Korpusebene arbeitet.
# Ausgabe ist die Bewertung eines jeden Satzes. Zur Bewertung des Korpus kann anschließend der Mittelwert gebildet werden.
!python3 sentence-meteor.py Data/Europarl.de-en.de-filtered.de.subword.test.desubword Output/pred_batch_size_2k_60k.txt.desubword

In [ ]:
# Evaluierung mit Word Error Rate (WER)
!python3 corpus-wer.py Data/Europarl.de-en.de-filtered.de.subword.test.desubword Output/pred_batch_size_2k_60k.txt.desubword